# 🧠 Dataset Condensation via Distribution Matching (PyTorch Implementation)

### 📘 Primer

This notebook implements **Dataset Condensation using Distribution Matching (DM)** — a data-efficient learning approach that synthesizes a *compact yet representative* set of samples capable of training models nearly as well as the full dataset.

---

### 🧩 What is Dataset Condensation?

Dataset Condensation aims to **compress a large dataset into a much smaller synthetic set** that preserves the original data’s learning signal.  
Instead of storing millions of real examples, we learn a small synthetic dataset `x_syn, y_syn` such that models trained on it achieve similar generalization.

In this implementation, we use **Distribution Matching (DM)** — a technique that aligns feature distributions between real and synthetic data in the output space of a neural network.  
At each iteration, synthetic data are updated to minimize the discrepancy between the mean activations of real and synthetic examples.

---

### 🚀 Workflow Overview

1. **Setup & Reproducibility**
   - Import required libraries.
   - Define deterministic seeds for Python, NumPy, and PyTorch.
   - Set key experiment hyperparameters (`ipc`, `eval_net_type`, `optim`, etc.).

2. **Data Loading**
   - Load and preprocess the time-series dataset (train/val/test splits).
   - Infer model input dimensions from a single training batch.

3. **Synthetic Data Initialization**
   - Initialize `x_syn` and `y_syn` either from random noise (`init='rand'`) or sampled real examples (`init='real'`).
   - Attach gradients to `x_syn` so it can be optimized directly.

4. **Baseline Evaluation**
   - Train a baseline model (LSTM/RNN) on the initial synthetic data.
   - Compute validation and test performance (AUC/APR) before condensation.

5. **Distribution Matching Loop**
   - For each iteration:
     - Randomly sample real data from each class.
     - Pass real and synthetic data through a frozen random network.
     - Compute **DM loss** = squared distance between mean feature activations.
     - Backpropagate through synthetic data and update `x_syn` via SGD.
   - Periodically evaluate on validation data and save the best synthetic set.

6. **Final Evaluation**
   - Load the best condensed dataset (`x_syn`, `y_syn`).
   - Train a fresh model from scratch on this condensed dataset.
   - Evaluate on validation and test sets to measure performance retention.

---

### ⚙️ Key Parameters

| Parameter | Description | Example |
|------------|-------------|----------|
| `ipc` | Instances per class to synthesize | `100` |
| `init` | Initialization mode (`rand` or `real`) | `'rand'` |
| `optim` | Optimizer for condensation (`sgd` / `adam`) | `'sgd'` |
| `eval_it` | Frequency (in iterations) for validation evaluation | `250` |
| `Iteration` | Total number of condensation iterations | `5000` |
| `eval_net_type` | Evaluation model architecture | `'lstm'` or `'rnn'` |

---

### 🧪 Objective

By the end of training, you’ll obtain a **synthetic dataset** that:
- Drastically reduces data size,
- Retains predictive power,
- Can be reused to train new models efficiently.

This forms the basis for scalable data-efficient learning and privacy-preserving model training — useful in resource-limited or federated settings.

---

> **References**
> - Zhao & Bilen (2021), *Dataset Condensation with Gradient Matching*  
> - Zhao et al. (2023), *Dataset Condensation via Distribution Matching*
> - PyTorch-based open-source reimplementations

---



---

## 🎯 Exercises — Explore and Compare

Use these guided experiments to deepen your understanding of **Distribution Matching for Dataset Condensation**.

1. **Vary the number of instances per class (`ipc`)**
   - Change `ipc` in the setup cell (e.g. `ipc = 500`, `200`, `50`, `10`).
   - Re-run the experiment and note:
     - How does validation/test **AUC** change as `ipc` decreases?
     - How does training time scale with smaller synthetic datasets?

2. **Change the initialization strategy**
   - Switch between  
     - `init = 'rand'` → purely random noise,  
     - `init = 'real'` → sampled from true data.
   - Observe how the **convergence speed** and **final synthetic quality** differ.

3. **Compare models trained on real vs. synthetic data**
   - From **Exercise 1**, you already trained a baseline model on the **real dataset**.
   - Now, compare its test performance with the model trained here on **synthetic data**.
   - Discuss:
     - How close does the synthetic dataset come to the real dataset’s performance?
     - Which metrics (AUROC, AUPRC) show the largest gap?

4. **Change evaluation architecture**
   - Try `eval_net_type = 'rnn'` vs. `'lstm'` (or even a CNN/MLP if applicable).
   - Does a more expressive model help capture patterns from condensed data?

5. **Inspect learned synthetic samples**
   - Visualize `x_syn` for each class (plot feature distributions or sample sequences).
   - Do the synthetic examples resemble the original data’s temporal structure?

---

🧭 **Goal:**  
By the end of these exercises, you should understand:
- How data condensation quality depends on `ipc` and initialization.  
- The trade-off between dataset size and generalization.  
- How well condensed data generalize across different model architectures.

---


In [ ]:
# ### 🔧 Setup: Reproducibility and Imports
# 
# This cell:
# - Imports all required libraries.
# - Defines a `set_seed()` utility to make runs deterministic.
# - Ensures consistent results across NumPy, Python, and PyTorch.

# %%
import os
import random
import numpy as np
import torch
from tqdm import tqdm
from models import get_net
# project-specific helpers
from utils_1 import *
from loaders import *
import warnings
warnings.filterwarnings("ignore")  # Ignore all warnings




def set_seed(seed: int = 42) -> None:
    """
    Ensure full reproducibility across Python, NumPy, and PyTorch.
    This helps compare condensation runs fairly.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Make CuDNN (PyTorch backend) deterministic
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    os.environ["PYTHONHASHSEED"] = str(seed)

# Set random seed once for the whole notebook
SEED = 42
set_seed(SEED)
print(f"✅ Random seed set to {SEED}")

In [ ]:
# 
# `ipc` = *instances per class* to synthesize.  
# Adjust these to control how much synthetic data is generated.

# %%
ipc = 100             # instances per class to synthesize (e.g. 500, 200, 100, 50, 10)
init = 'rand'        # initialization: 'real' (sampled real data) or 'rand' (random noise)
optim = 'sgd'        # optimizer for condensation: 'sgd' or 'adam'
eval_net_type = 'lstm'  # model type for evaluation: 'rnn' or 'lstm' (we will train this model on synthesised data for evaluation)

print(f"ipc={ipc}, init='{init}', optim='{optim}', eval_net_type='{eval_net_type}'")
device = get_device()
print(f"Using device: {device}")

In [3]:
class Args:
    lr_x = 100
    mom_x = 0.99
    Iteration = 5000
    batch_real = 256
    eval_it = 250
args = Args()

In [ ]:
# load data
train_batch = 256

# -------------------------
# Load data (adjust `path` to where your pickles are)
# -------------------------
data_path = "../DATA/"   # <-- change to your data folder
train_loader, val_loader, test_loader = get_loaders_time_series(
    path=data_path,
    train_batch=train_batch,
    val_batch=256,
    test_batch=256,
    sampler=True,
    pre_process="std",
    ds_half=0,
)

# Inspect one batch shape to infer model input dims
batch_x, batch_y = next(iter(train_loader))
print("One batch x shape:", batch_x.shape, "y shape:", batch_y.shape)
# typical shape: (batch, seq_len, n_features)

seq_len = batch_x.shape[1]
n_features = batch_x.shape[2]
print(f"seq_len={seq_len}, n_features={n_features}")

In [ ]:
# Initialize synthetic data
if init == 'rand':
    print('Initializing condensed data and labels from random noise data')
    x_syn, y_syn, _ = initialize_synthetic_data(train_loader, ipc, device, rand_real=True)

else:
    print('Initializing condensed data from random real data and binary labels')
    x_syn, y_syn, _ = initialize_synthetic_data(train_loader, ipc, device, real=True)

In [ ]:

# ### 🧩 Baseline performance
# Train and evaluate a baseline model on synthetic data (x_syn, y_syn)
# using the chosen network type and optimizer.

# %%
# Initialize baseline network
baseline_net = get_net(net_type=eval_net_type, input_dim=n_features, device=device).to(device)

# Binary classification loss (expects logits as model outputs)
criterion = nn.BCEWithLogitsLoss()

# Train and evaluate on validation set
# Returns: validation AUC, (val_loss ignored here), validation APR, and trained model
val_auc, _, val_apr, baseline_net = train_and_evaluate(
    baseline_net,
    x_syn, y_syn,
    val_loader,
    device,
    criterion,
    optim=optim,      # optimizer type: 'sgd' or 'adam'
    lr=0.0001,        # learning rate
    epochs=25,        # number of training epochs
    mom=0.9           # momentum (for SGD)
)

print(f"Val AUC/AUPRC : {val_auc:.4f}/{val_apr:.4f}")

# Evaluate trained model on test set
test_auc, _, test_apr = prediction_binary(baseline_net, test_loader, criterion, device)
print(f"Test AUC/AUPRC: {test_auc:.4f}/{test_apr:.4f}")


In [7]:
# Make sure x_syn, y_syn are on the correct device and have appropriate grad requirements
x_syn = x_syn.to(device)
y_syn = y_syn.to(device)
x_syn.requires_grad = True

optimizer_x = torch.optim.SGD([x_syn], lr=args.lr_x, momentum=args.mom_x)

In [8]:
# build the original train data getter: get random n data from class c
num_classes = 2
tr_data, tr_lb = get_data_array(train_loader)
get_data = build_data_getter(tr_data, tr_lb, device)

In [ ]:
# Improved, commented version of your training / condensation loop

import os
import torch

# --- Initialization / bookkeeping ---
best = -float("inf")          # best validation AUC seen so far (use -inf to allow any first result)
loss_sum = 0.0                # running sum of DM loss across iterations (for printing / logging)
iters_since_eval = 0          # counts iterations since last evaluation (helps averaging)
best_x_syn = None
best_y_syn = None

# Learning loop
for it in range(args.Iteration):
    iters_since_eval += 1

    # ---- Periodic evaluation (run every args.eval_it iterations) ----
    if (it + 1) % args.eval_it == 0:

        # Create a fresh evaluation network (same architecture/type you use for eval).
        # Move it to the correct device and ensure it's in evaluation mode inside train_and_evaluate.
        eval_net = get_net(net_type=eval_net_type,
                           input_dim=n_features,
                           device=device).to(device)

        # Train/evaluate on validation set (train_and_evaluate should handle .train/.eval internally).
        # We pass clones of synthetic tensors to avoid accidental in-place changes.
        val_auc, _, val_apr, _ = train_and_evaluate(
            eval_net,
            x_syn.clone(),
            y_syn.clone(),
            val_loader,
            device,
            criterion,
            lr=0.001,
            epochs=50,
            optim=optim,
            mom=0.9
        )

        # Average DM loss across the iterations since last eval for logging.
        avg_dm_loss = loss_sum / max(1, iters_since_eval)

        print(
            f"Itr {it + 1}, "
            f"Val AUC: {val_auc:.4f}, "
            f"Val AUPRC: {val_apr:.4f}, "
            f"DM Loss (avg over last {iters_since_eval} iters): {avg_dm_loss:.5f}"
        )

        # Reset the running loss counters for the next evaluation window.
        loss_sum = 0.0
        iters_since_eval = 0

        # If validation improved, save the current synthetic dataset (deep-copied to CPU).
        if val_auc > best:
            best = val_auc
            best_x_syn = x_syn.clone().detach().cpu()
            best_y_syn = y_syn.clone().detach().cpu()

            # ensure the target directory exists
            os.makedirs('./synthetic_data/', exist_ok=True)
            torch.save({
                "x_syn": best_x_syn,
                "y_syn": best_y_syn,
            }, os.path.join('./synthetic_data/', f'{init}_{ipc}.pth'))

    # ---- Learning / updating the synthetic set for this iteration ----
    # Create a fresh random network (same architecture you use for condensation)
    net = get_net(input_dim=n_features, output_dim=1, device=device).to(device)
    net.train()  # ensure train-mode if there are batchnorm/dropout layers

    # We do not want gradients with respect to model parameters (only w.r.t. synthetic data),
    # so freeze network parameters. Gradients must flow to synthetic tensors (x_syn).
    for param in net.parameters():
        param.requires_grad = False

    # Compute the distribution-matching (dm) loss across classes.
    # Start as a zero-tensor on the device so backward() works correctly.
    dm_loss = torch.tensor(0.0, device=device)

    # For each class, compare the network outputs on real vs synthetic examples.
    # We average/accumulate per-class squared difference of means in output space.
    for c in range(num_classes):
        # Determine how many real examples to sample for class c this iteration.
        this_batch_real = min(len(get_data.indices_class[c]), args.batch_real)
        # Sample from the real dataset for class c. Make sure it requires grad if you ever
        # want gradients flowing to inputs (not typically necessary for pure DM loss).
        batch_data_real = get_data(c, this_batch_real).to(device).requires_grad_(False)

        # Select the corresponding slice of synthetic data for class c.
        # x_syn is assumed to be arranged per-class in contiguous blocks:
        #            x_syn[c * ipc : (c + 1) * ipc]
        batch_data_syn = x_syn[c * ipc: (c + 1) * ipc].to(device)
        batch_data_syn.requires_grad_(True)   # we want gradients w.r.t. the synthetic examples

        # Forward through the frozen net.
        # Detach the real outputs so they act as constants when comparing to synthetic outputs.
        output_real = net(batch_data_real).detach()
        output_syn = net(batch_data_syn)  # gradients will flow back to batch_data_syn

        # Compare the mean network outputs per class (MSE between means).
        # If output is batch x 1, torch.mean(..., dim=0) will produce shape (1,) so make sure shapes match.
        mean_diff = torch.mean(output_real, dim=0) - torch.mean(output_syn, dim=0)
        dm_loss += torch.sum(mean_diff ** 2)

    # Update synthetic data using optimizer_x (assumed to optimize x_syn).
    optimizer_x.zero_grad()
    dm_loss.backward()
    optimizer_x.step()

    # Accumulate normalized loss for logging: divide by num_classes to report per-class average.
    loss_sum += dm_loss.item() / float(num_classes)


In [ ]:
import os
import torch

# ---- Final evaluation of model trained on learned condensed dataset ----

EPOCHS = 100
device = device  # assume device is already set (e.g., "cuda" or "cpu")

# Create model and optimizer (RNN here — keep consistent with training/eval nets).
model = get_net(net_type='lstm', hidden_dim=32,input_dim=n_features, device=device).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Path to saved condensed dataset
syn_path = os.path.join('./synthetic_data/', f'{init}_{ipc}.pth')
if not os.path.exists(syn_path):
    raise FileNotFoundError(f"Condensed dataset not found at {syn_path}")

# Load checkpoint (maps tensors to current device)
checkpoint = torch.load(syn_path, map_location=device)
best_x_syn = checkpoint["x_syn"]
best_y_syn = checkpoint["y_syn"]

# Move to device and ensure proper dtypes
image_syn_eval = best_x_syn.to(device)
label_syn_eval = best_y_syn.to(device)

# Determine a sensible batch size for evaluation training
syn_batch = int(min(256, len(image_syn_eval)))  # use length of loaded tensors

# Build dataloader for synthetic dataset
synthetic_dataset = torch.utils.data.TensorDataset(image_syn_eval, label_syn_eval)
synthetic_loader = torch.utils.data.DataLoader(
    synthetic_dataset, batch_size=syn_batch, shuffle=True, drop_last=False
)

# Logging / best validation tracker
best_val_auc = -float("inf")
os.makedirs('./checkpoints', exist_ok=True)  # ensure directory exists for model saves

# Optional: progress bar (commented out). Uncomment if tqdm is available.
# from tqdm import trange, tqdm
# epoch_iter = trange(EPOCHS, desc="Final Eval")
# for epoch in epoch_iter:

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    # Train one epoch on the synthetic condensed dataset
    for x_batch, y_batch in synthetic_loader:
        # Ensure types and device; some nets expect float32 input.
        x_batch = x_batch.to(torch.float32).to(device)
        y_batch = y_batch.to(torch.float32).to(device)

        optimizer.zero_grad()
        preds = model(x_batch)

        # If model returns (B,1), flatten to (B,) for many losses; leave as-is if already (B,)
        if preds.dim() == 2 and preds.size(1) == 1:
            preds = preds[:, 0]

        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Average training loss per batch (safeguard if loader is empty)
    avg_train_loss = running_loss / max(1, len(synthetic_loader))

    # ---------- Validation ----------
    # prediction_binary should evaluate model on val_loader and return (loss, auc, apr)
    val_loss, val_auc, val_apr = prediction_binary(model, val_loader, criterion, device)

    print(
        f"Epoch {epoch+1}/{EPOCHS} — Train loss: {avg_train_loss:.4f} | "
        f"Val loss: {val_loss:.4f} | Val AUC: {val_auc:.4f} | Val AUPRC: {val_apr:.4f}"
    )

    # Save best model (save state_dict + metadata rather than whole model object)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        save_path = os.path.join('./checkpoints', 'dc_model_best.pth')
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_auc': val_auc,
            'val_apr': val_apr,
            'train_loss': avg_train_loss,
        }, save_path)
        print(f"  -> New best Val AUC: {val_auc:.4f}. Saved checkpoint to {save_path}")

# After training, if you want to load the best model weights back into `model`, you can:
# ckpt = torch.load('./checkpoints/dc_model_best.pth', map_location=device)
# model.load_state_dict(ckpt['model_state_dict'])


In [ ]:
dict=torch.load('./checkpoints/dc_model_best.pth')
model.load_state_dict(dict['model_state_dict'])
test_loss, test_auc, test_apr = prediction_binary(model, test_loader, criterion, device)
print(f"Test loss: {test_loss:.4f} | Test AUC: {test_auc:.4f} | Test AUPRC: {test_apr:.4f}")